In [ ]:
import torch
import non_local_boxes
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

import warnings  
warnings.filterwarnings("ignore")  # to ignore the warning messages

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

# in ordert to have unblurred pictures
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

In [ ]:
non_local_boxes.evaluate.nb_columns = 1

---
# I. Definitions

### Nonlocal Boxes

In [ ]:
matrix_to_tensor = non_local_boxes.utils.matrix_to_tensor
tensor_to_matrix = non_local_boxes.utils.tensor_to_matrix

# Boxes as 2x2x2x2 tensors
PR = matrix_to_tensor(non_local_boxes.utils.PR)
PRprime = matrix_to_tensor(non_local_boxes.utils.PRprime)
SR = matrix_to_tensor(non_local_boxes.utils.SR)
I = matrix_to_tensor(non_local_boxes.utils.I)
P0 = matrix_to_tensor(non_local_boxes.utils.P_0)
P1 = matrix_to_tensor(non_local_boxes.utils.P_1)

def WinCHSH(P): return float(non_local_boxes.evaluate.h_flat(P))                    # P is a 2x2x2x2 tensor
def WinCHSHprime(P): return float(non_local_boxes.evaluate.h_prime_flat(P))         # P is a 2x2x2x2 tensor
def BoxProduct(W,P,Q): return non_local_boxes.evaluate.R_tensor(W,P,Q)[:,:,:,:,0]   # P,Q are is a 2x2x2x2 tensor

### Test if a Box $\mathtt{P}$ is in $\mathcal Q$

To draw $\mathcal Q$, we use this article: https://arxiv.org/abs/quant-ph/0309137

In [ ]:
def Compute_Coeff(P1, P2, P3, G_value, CHSH_value, WinGame):
    A = np.array([[WinGame(P1), WinGame(P2), WinGame(P3)],
                  [WinCHSH(P1), WinCHSH(P2), WinCHSH(P3)],
                  [1, 1, 1]])
    b = np.array([G_value, CHSH_value, 1])
    return np.linalg.solve(A,b).tolist()

def E(Box, x, y):
    return Box[0,0,x,y] + Box[1,1,x,y] - Box[0,1,x,y] - Box[1,0,x,y]

def is_in_Q(G_value,CHSH_value, Box1, Box2, Box3, WinGame):
    alpha, beta, gamma = Compute_Coeff(Box1, Box2, Box3, G_value, CHSH_value, WinGame)
    Box = alpha*Box1 + beta*Box2 + gamma*Box3
        # Box is a 2x2x2x2 tensor
    coeff00 = np.arcsin(E(Box,0,0))
    coeff01 = np.arcsin(E(Box,0,1))
    coeff10 = np.arcsin(E(Box,1,0))
    coeff11 = np.arcsin(E(Box,1,1))
    return coeff00 + coeff01 + coeff10 - coeff11 <= np.pi

### Computation of Orbit $(\mathtt{P})$

In [ ]:
def Compute_Orbits(P, W, max_depth):
    Orbits = [[P]]  # each all_orbits[k] is a list, which is the orbit of depth k+1
    for k in range(max_depth-1):
          Orbits.append([])
          for l in range(k+1):
                for Q1 in Orbits[l]:
                      for Q2 in Orbits[k-l]:
                            Orbits[k+1].append( BoxProduct(W, Q1, Q2) )
    return Orbits

In [ ]:
def Compute_Orbit_BS09(P, W, max_depth):
    OrbitBS09 = [P]
    k = 1
    while 2**k<=max_depth:
        k += 1
        Q = OrbitBS09[-1]
        OrbitBS09.append( BoxProduct(W, Q, Q) )
    return OrbitBS09

In [ ]:
def Draw_Orbit(P, P1, P1name, P2, P2name, P3, P3name, WinGame, GameName, W, Wname, max_depth, details_BS09=False, precision_Q=0.0001, big=True, position_legend='upper right', left_limit=0.):
    # P, P1, P2, P3 are 2x2x2x2 matrices

    if non_local_boxes.evaluate.nb_columns!=1:
        print("\n   WARNING: Please, set the number of columns to 1 (in `non_local_boxes.evaluate`).")
        return None

    # Draw the initial background triangle:
    x1, y1 = WinGame(P1), WinCHSH(P1)
    x2, y2 = WinGame(P2), WinCHSH(P2)
    x3, y3 = WinGame(P3), WinCHSH(P3)

    # Check if our coordinate system is good, i.e. if projected points aren't aligned:
    if (x1 - x3)*(y2-y3)==(x2-x3)*(y1-y3):
            print("\n    WARNING: Error in the coordinate system.")
            return None

    triangle = plt.Polygon(np.array([[x1, y1], [x2,y2], [x3, y3]]), color=hsv_to_rgb([0.475, 1, 0.9]))
    plt.gca().add_patch(triangle)

    BBLMTU_value = (3+math.sqrt(6))/6
    BBLMTU_rectangle = plt.Polygon(np.array([[x1, y1], [x1+(x2-x1)*(1-BBLMTU_value)/(1-y2),BBLMTU_value], [x1 - (x1-x3)*(1-BBLMTU_value)/(1-y3), BBLMTU_value]]), color=hsv_to_rgb([0.475, 1, 0.5])) # use Thales theorem
    plt.gca().add_patch(BBLMTU_rectangle)

    # Draw the Quantum area
    G_value=min(x2,x3)
    CHSH_value=(2+np.sqrt(2))/4
    Quantum_color = 'pink'
    while G_value<=max(x2, x3):
        if is_in_Q(G_value, CHSH_value, P1, P2, P3, WinGame):
            plt.plot([G_value, G_value], [CHSH_value, min(y2, y3)], color=Quantum_color)
            G_value+=precision_Q
        else:
            CHSH_value= max(CHSH_value-precision_Q, min(y2, y3))
    #plt.plot(0.75, 0.75,'o', markersize=0.7, color=Quantum_color)
    plt.plot([min(x2,x3), max(x2, x3)], [y2,y2], linewidth=3, color=Quantum_color)
    plt.plot([min(x2,x3), max(x2, x3)], [y2,y2], "o", markersize=5, color=Quantum_color)

    # Draw the orbit:
    Orbits = Compute_Orbits(P, W, max_depth)
    for k in range(len(Orbits)):
        color = hsv_to_rgb([k/(7*(len(Orbits)-1)), 1, 1])
        for Q in Orbits[k]:  # Q is a tensor
            # Compute the abscissa and the ordinate:
            x = WinGame(Q)
            if x>=left_limit:
                y = WinCHSH(Q)
                plt.plot(x, y, 'o', markersize=4, color=color)
                if k==2-1 and details_BS09:
                     plt.text(x, y, "  ← orbit of depth $k=2$", horizontalalignment='left', verticalalignment = 'center')
                     
                     
    # Draw the orbit from [BS09]:
    if details_BS09:
        OrbitBS09 = Compute_Orbit_BS09(P, W, max_depth)
        for Q in OrbitBS09:
            x = WinGame(Q)
            if x>=left_limit:
                y = WinCHSH(Q)
                plt.plot(x, y, 'o', markersize=4, mfc='none', color='black')
    
    plt.text(WinGame(P), WinCHSH(P), "  ← starting box $\mathbf{P}$", horizontalalignment='left', verticalalignment = 'center')
    plt.text(x1+0.001, y1, P1name, horizontalalignment='center',weight="bold")
    plt.text(x2, y2-0.007, P2name, verticalalignment='center',weight="bold")
    plt.text(x3, y3-0.007, P3name, verticalalignment='center',weight="bold")
    if big:  fontsize=15
    else:    fontsize=13
    plt.title("Orbit of a box $\mathbf{P}$ until depth $k=$"+str(max_depth)+" (wiring $\mathsf{W}=$"+Wname+")", fontsize=fontsize)
    if big:  fontsize=12
    else: fontsize=10
    plt.xlabel("$\mathbb{P}($win at "+GameName+")", fontsize=fontsize)
    plt.ylabel("$\mathbb{P}($win at $\mathbf{CHSH}$)", fontsize=fontsize)
    handles = []
    if big:  label='Collapsing $\mathcal{N\!S}$-boxes\nfrom [BBLMTU06].'
    else:  label='Collapsing $\mathcal{N\!S}$-boxes from [BBLMTU06].'
    handles.append( mpatches.Patch(color=hsv_to_rgb([0.475, 1, 0.5]), label=label) )
    handles.append( mpatches.Patch(color=hsv_to_rgb([0.475, 1, 0.8]), label='Other $\mathcal{N\!S}$-boxes.') )
    handles.append( mpatches.Patch(color=Quantum_color, label='Quantum boxes.') )
    handles.append( mlines.Line2D([], [],color=hsv_to_rgb([0.0, 1, 1]), marker='o',ls='', label='Orbit of depth $k=$'+str(1)+".") )
    handles.append(  mlines.Line2D([], [],color=hsv_to_rgb([1/14, 1, 1]), marker='o',ls='', label='Orbit of depth $k=$'+str(max_depth//2)+".") )
    handles.append( mlines.Line2D([], [],color=hsv_to_rgb([1/7, 1, 1]), marker='o',ls='', label='Orbit of depth $k=$'+str(max_depth)+".") )
    if details_BS09:
        handles.append( mlines.Line2D([], [],color='black', mfc='none', marker='o',ls='', label='Boxes studied in [BS09].') )
    if big: fontsize=12
    else:   fontsize=9
    plt.legend(handles=handles, loc=position_legend, fontsize = fontsize)
    ax=plt.gca()
    ax.relim(), ax.autoscale_view()   # auto-scale axes
    plt.show()

---
# II. Draw the Orbit (in the slice $\mathtt{PR}-\mathtt{SR}-\mathtt{I}$)

In [ ]:
alpha, beta, gamma = Compute_Coeff(PR, SR, I, G_value=0.627, CHSH_value=0.862, WinGame=WinCHSHprime)
P = alpha*PR + beta*SR + (1-alpha-beta)*I

m = non_local_boxes.evaluate.nb_columns

In [ ]:
W = non_local_boxes.utils.W_BS09(m)

Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = SR,
    P2name = "SR",
    P3 = (I+PR)/2,
    P3name = "(I+PR)/2",
    WinGame=WinCHSHprime,
    GameName="$\mathbf{CHSH}'$",
    W = W,
    Wname = "$\mathsf{W}_{BS}$",
    max_depth = 6, 
    details_BS09=True,
    precision_Q=0.0001,
    big=False,
    left_limit=0.5
)

In [ ]:
W = non_local_boxes.utils.W_FWW09(m)
non_local_boxes.utils.print_functions_from_wiring(W)

Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = SR,
    P2name = "SR",
    P3 = (I+PR)/2,
    P3name = "(I+PR)/2",
    WinGame=WinCHSHprime,
    GameName="$\mathbf{CHSH}'$",
    W = W,
    Wname = "$\mathsf{W}_{\oplus}$",
    max_depth = 6,
    precision_Q=0.0001,
    left_limit=0.5
)

In [ ]:
W =   [0.,0.,1.,1.,0.,0.,1.,1.,0.,0.,0.,1.,0.,0.,0.,1.,1.,0.,0.,1.,1.,0.,0.,1.,1.,0.,0.,1.,1.,0.,0.,1.]
non_local_boxes.utils.print_functions_from_wiring(W)
W = torch.tensor(W)
W = torch.t(W.repeat(m, 1))

Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = SR,
    P2name = "SR",
    P3 = (I+PR)/2,
    P3name = "(I+PR)/2",
    WinGame=WinCHSHprime,
    GameName="$\mathbf{CHSH}'$",
    W = W,
    Wname = "$\mathsf{W}_{(b)}$",
    max_depth = 6,
    precision_Q=0.0001,
    left_limit=0.5
)

In [ ]:
W =    [0.,0.,1.,1.,0.,0.,1.,1.,0.,0.,1.,0.,0.,0.,1.,0.,1.,0.,0.,1.,1.,0.,0.,1.,1.,0.,0.,1.,1.,0.,0.,1.]
non_local_boxes.utils.print_functions_from_wiring(W)
W = torch.tensor(W)
W = torch.t(W.repeat(m, 1))

Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = SR,
    P2name = "SR",
    P3 = (I+PR)/2,
    P3name = "(I+PR)/2",
    WinGame=WinCHSHprime,
    GameName="$\mathbf{CHSH}'$",
    W = W,
    Wname = "$\mathsf{W}_{(c)}$",
    max_depth = 6,
    precision_Q=0.0001,
    left_limit=0.5
)

In [ ]:
W = torch.tensor(non_local_boxes.utils.known_collapsing_W[16] )*1.
non_local_boxes.utils.print_functions_from_wiring(W)
W = torch.tensor(W)
W = torch.t(W.repeat(m, 1))

Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = SR,
    P2name = "SR",
    P3 = (I+PR)/2,
    P3name = "(I+PR)/2",
    WinGame=WinCHSHprime,
    GameName="$\mathbf{CHSH}'$",
    W = W,
    Wname = "$\mathsf{W}_{(d)}$",
    max_depth = 6,
    precision_Q=0.0001,
    left_limit=0.5
)

------
# III. Drawing the Orbit (in the slice $\mathtt{PR}-\mathtt{P}_0-\mathtt{P}_1$)

We need to define another game to study an orthogonal slice to the previous one:

In [ ]:
def MyGame(P):
    # P is a 2x2x2x2 tensor
    game = torch.zeros((2,2,2,2))
    for a in range(2):
        for b in range(2):
            for x in range(2):
                for y in range(2):
                    if a == 0 and b == y:
                        game[a,b,x,y]=0.25
    return float(torch.sum(P*game))

Let's go:

In [ ]:
alpha, beta = 0.28, 0.7
P = alpha*PR + beta*P0 + (1-alpha-beta)*P1

W = torch.tensor(non_local_boxes.utils.known_collapsing_W[0] )*1.
non_local_boxes.utils.print_functions_from_wiring(W)
W = torch.tensor(W)
W = torch.t(W.repeat(m, 1))

Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = P0,
    P2name = "P0",
    P3 = P1,
    P3name = "P1",
    WinGame=MyGame,
    GameName="$\mathbf{G}$",
    W = W,
    Wname = "$\mathsf{W}_{BS}$",
    max_depth = 6,
    precision_Q=0.0001,
    position_legend='upper left'
)

In [ ]:
alpha, beta = 0.2, 0.05
P = alpha*PR + beta*P0 + (1-alpha-beta)*P1

W = torch.tensor(non_local_boxes.utils.known_collapsing_W[0] )*1.
non_local_boxes.utils.print_functions_from_wiring(W)
W = torch.tensor(W)
W = torch.t(W.repeat(m, 1))

Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = P0,
    P2name = "P0",
    P3 = P1,
    P3name = "P1",
    WinGame=MyGame,
    GameName="$\mathbf{G}$",
    W = W,
    Wname = "$\mathsf{W}_{BS}$",
    max_depth = 6,
    precision_Q=0.0001,
    position_legend='upper left'
)

In [ ]:
alpha, beta = 0.25, 0.7
P = alpha*PR + beta*P0 + (1-alpha-beta)*P1

non_local_boxes.utils.print_functions_from_wiring(W)
W = torch.tensor(W)
W = torch.t(W.repeat(m, 1))
W=non_local_boxes.utils.W_FWW09(1)

Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = P0,
    P2name = "P0",
    P3 = P1,
    P3name = "P1",
    WinGame=MyGame,
    GameName="$\mathbf{G}$",
    W = W,
    Wname = "$\mathsf{W}_{\oplus}$",
    max_depth = 6,
    precision_Q=0.0001,
    position_legend='upper left'
)

In [ ]:
alpha, beta = 0.25, 0.7
P = alpha*PR + beta*P0 + (1-alpha-beta)*P1

W = torch.tensor(non_local_boxes.utils.known_collapsing_W[27] )*1.
non_local_boxes.utils.print_functions_from_wiring(W)
W = torch.tensor(W)
W = torch.t(W.repeat(m, 1))

Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = P0,
    P2name = "P0",
    P3 = P1,
    P3name = "P1",
    WinGame=MyGame,
    GameName="$\mathbf{G}$",
    W = W,
    Wname = "$\mathsf{W}_{(b)}$",
    max_depth = 6,
    precision_Q=0.0001,
    position_legend='upper left'
)

In [ ]:
alpha, beta = 0.25, 0.7
P = alpha*PR + beta*P0 + (1-alpha-beta)*P1

W = torch.tensor(non_local_boxes.utils.known_collapsing_W[21] )*1.
non_local_boxes.utils.print_functions_from_wiring(W)
W = torch.tensor(W)
W = torch.t(W.repeat(m, 1))

Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = P0,
    P2name = "P0",
    P3 = P1,
    P3name = "P1",
    WinGame=MyGame,
    GameName="$\mathbf{G}$",
    W = W,
    Wname = "$\mathsf{W}_{(c)}$",
    max_depth = 6,
    precision_Q=0.0001,
    position_legend='upper left'
)

In [ ]:
alpha, beta = 0.25, 0.7
P = alpha*PR + beta*P0 + (1-alpha-beta)*P1

W = torch.tensor(non_local_boxes.utils.known_collapsing_W[16] )*1.
non_local_boxes.utils.print_functions_from_wiring(W)
W = torch.tensor(W)
W = torch.t(W.repeat(m, 1))

Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = P0,
    P2name = "P0",
    P3 = P1,
    P3name = "P1",
    WinGame=MyGame,
    GameName="$\mathbf{G}$",
    W = W,
    Wname = "$\mathsf{W}_{(d)}$",
    max_depth = 6,
    precision_Q=0.0001,
    position_legend='upper left'
)